In [ ]:
#Incomplete

import pandas as pd
import numpy as np
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from sklearn.ensemble import VotingRegressor
from sklearn import linear_model
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import copy

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer, OneHotEncoder
from sklearn.metrics import make_scorer

import timeit
import time

#Section A: preprocessing

#function to check times at different points in code
times = []
tstart = time.time()
def how_long(t_st):
    t1 = time.time()
    td = t1 - t_st
    times.append(td)
    print('Part {} time {}'.format(len(times), td))
    return t1

#reading data files
trainRaw = pd.read_csv('train.csv')
testRaw = pd.read_csv('test.csv')
train = trainRaw.copy()
test = testRaw.copy()

# print(train.head())
# print(test.head())

#preprocessing
def preproc(df):
    #replace '4+' with 4 to make all Stay in City values numeric
    df['Stay_In_Current_City_Years'].replace('4+', 4, inplace=True)
    
    missing = df[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']]
    missing = missing.applymap(str)
    missing = missing.replace('nan', 'missing')
    df[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']] = missing
    
    #use ordinal encoder on Gender and Age columns
    ages = ['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+']
    mapAge = list(zip(ages, range(0,7)))
    dictGend = {'col': 'Gender', 'mapping': [('M', 0), ('F', 1)]}
    oe = ce.OrdinalEncoder(cols=['Gender', 'Age'], mapping=[dictGend, {'col': 'Age', 'mapping': mapAge}])
    df = oe.fit_transform(df)
    
    #use binary encoder on categorical features that should not be ordered
    catsNotOrd = ['User_ID', 'Product_ID', 'Occupation', 'City_Category', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']
    ceb = ce.BinaryEncoder(cols=catsNotOrd)
    df = ceb.fit_transform(df)
    
    #minmax scale Age
    scale = MinMaxScaler()
    df[['Age', 'Stay_In_Current_City_Years']] = scale.fit_transform(df[['Age', 'Stay_In_Current_City_Years']].values)
    
    return df

train = preproc(train)
test = preproc(test)

#making target and feature data sets
x = train.drop('Purchase', axis=1)
y = train['Purchase']

# Trying not using some features that seem to be less important
few_feat = ['Product_ID_1', 'Product_ID_2', 'Product_ID_3', 'Product_ID_4',
       'Product_ID_5', 'Product_ID_7', 'Product_ID_8', 'Product_ID_9',
       'City_Category_2', 'Product_Category_1_1', 'Product_Category_1_2',
       'Product_Category_1_3', 'Product_Category_1_4', 'Product_Category_1_5',
       'Age', 'Stay_In_Current_City_Years']
x0 = x.copy()
test0 = test.copy()
x1 = x[few_feat]
test1 = test[few_feat]

#train-test split
xtr0, xtest0, ytr, ytest = train_test_split(x0, y, random_state=2)
xtr1, xtest1, ytr, ytest = train_test_split(x1, y, random_state=2)

lxtr = [xtr0, xtr1]
lxtest = [xtest0, xtest1]

endtime = how_long(tstart)


#Section B part 1 version 3: building and tuning models

tstart = time.time()

#root mean squared log error evaluation function
def rmsle(y_targ, y_pred):
    return np.sqrt(mean_squared_error(y_targ, y_pred))

#starting with Random Forest Regressor, tuning n_estimators
rfr0 = RandomForestRegressor(max_depth=16, min_samples_split=10.0**(-4), max_features=0.8, max_leaf_nodes=None, random_state=2,
                             min_samples_leaf=1, n_jobs=5)

rfr_par_grid_n_est0 = {'n_estimators' : range(110, 141, 5)}

gridsearchcv_rfr_n_est = GridSearchCV(rfr0, param_grid=rfr_par_grid_n_est0, cv=5, n_jobs=5)
gridsearchcv_rfr_n_est.fit(xtr0, ytr)
res_rfrnest = gridsearchcv_rfr_n_est.cv_results_
best_est_rfrnest = gridsearchcv_rfr_n_est.best_estimator_
best_par_rfrnest = gridsearchcv_rfr_n_est.best_params_
print([res_rfrnest, best_est_rfrnest, best_par_rfrnest])




#tuning models for balance between speed and performance

# def tune_svsp(modeltune, xtu, ytu, max_time, param_to_adj, list_adj_vals, dict_const_params):
#     print(str(modeltune))
#     print(param_to_adj)
#     mtime = 0
#     xtutr, xtutest, ytutr, ytutest = train_test_split(xtu, ytu, random_state=2)
#     tuning_results = pd.DataFrame(index=range(len(list_adj_vals)), columns=['param_val', 'tr_rmsle', 'test_rmsle', 'time'])
#     for i, par_val in enumerate(list_adj_vals):
#         if mtime > max_time:
#             break
#         dict_const_params[param_to_adj] = par_val
#         mod = modeltune(**dict_const_params)
#         timeA = time.time()
#         mod.fit(xtutr, ytutr)
#         pred_tr = mod.predict(xtutr)
#         pred_test = mod.predict(xtutest)
#         mtime = time.time() - timeA
#         print(mtime)
#         score_rmsle_tr = rmsle(ytutr, pred_tr)
#         score_rmsle_test = rmsle(ytutest, pred_test)
#         tuning_results.iloc[i] = {'param_val' : par_val, 'tr_rmsle' : score_rmsle_tr, 'test_rmsle' : score_rmsle_test,
#                                  'time' : mtime}
#     print(tuning_results)
#     return tuning_results

#Random Forest Regressor

#tuning n_estimators; ~120 seems good

# vals_n_est = [5,6,7,8,9,20,50]
# dpar = {'max_depth' : 15, 'min_samples_split' : 2, 'max_leaf_nodes' : 50, 'max_features' : 'auto', 'random_state' : 2,
#         'n_jobs' : 5, 'min_samples_leaf' : 0.15}
# rfr_tune_nest_res = tune_svsp(RandomForestRegressor, xtr0, ytr, 8, 'n_estimators', vals_n_est, dpar)
# vals_n_est = range(36,48)
# dpar = {'max_depth' : 25, 'min_samples_split' : 0.4, 'max_leaf_nodes' : 50, 'max_features' : 0.3, 'random_state' : 2,
#         'n_jobs' : 5, 'min_samples_leaf' : 0.15}
# rfr_tune_nest_res = tune_svsp(RandomForestRegressor, xtr1, ytr, 8, 'n_estimators', vals_n_est, dpar)
# vals_n_est = [5,7,9,20,50]
# rfr_tune_nest_res = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'n_estimators', vals_n_est, dpar)
# dpar = {'max_depth' : None, 'min_samples_split' : 2, 'max_leaf_nodes' : None, 'max_features' : 'auto', 'random_state' : 2,
#         'n_jobs' : 5, 'min_samples_leaf' : 1}
# vals_n_est = [130, 160, 190, 210]
# rfr_tune_nest_res1 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'n_estimators', vals_n_est, dpar)
# vals_n_est = [300, 400, 500]
# rfr_tune_nest_res2 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'n_estimators', vals_n_est, dpar)
# vals_n_est = [150, 170, 180]
# rfr_tune_nest_res2 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'n_estimators', vals_n_est, dpar)
# print([rfr_tune_nest_res, rfr_tune_nest_res1, rfr_tune_nest_res2])
# vals_n_est = [100, 110, 120]
# rfr_tune_nest_res3 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'n_estimators', vals_n_est, dpar)

#tuning max_depth; ~16 seems good
# dpar = {'n_estimators' : 130, 'min_samples_split' : 2, 'max_leaf_nodes' : None, 'max_features' : 'auto', 'random_state' : 2,
#         'n_jobs' : 5, 'min_samples_leaf' : 1}
# vals_md = [5, 10, 15, 100]
# rfr_tune_md_res0 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'max_depth', vals_md, dpar)
# vals_md = [13,14,15,16,17,25,40]
# rfr_tune_md_res1 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'max_depth', vals_md, dpar)
# print([rfr_tune_md_res0, rfr_tune_md_res1])
# vals_md = range(18,22)
# rfr_tune_md_res2 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'max_depth', vals_md, dpar)

#tuning min_samples_split; around 10.0**-4 seems good
# dpar = {'n_estimators' : 130, 'max_depth' : 16, 'max_leaf_nodes' : None, 'max_features' : 'auto', 'random_state' : 2,
#         'n_jobs' : 5, 'min_samples_leaf' : 1}
# vals_mss = [0.5,0.4,0.3,0.2]
# rfr_tune_mss_res0 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'min_samples_split', vals_mss, dpar)
# vals_mss = [0.15, 0.1, 0.05, 0.01]
# rfr_tune_mss_res1 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'min_samples_split', vals_mss, dpar)
# vals_mss = [0.005, 0.001, 2]
# rfr_tune_mss_res2 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'min_samples_split', vals_mss, dpar)
# vals_mss = [50,10,4,3]
# rfr_tune_mss_res3 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'min_samples_split', vals_mss, dpar)
# vals_mss = 5*(10.0**(-np.arange(4,8)))
# rfr_tune_mss_res4 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'min_samples_split', vals_mss, dpar)
# vals_mss = 10.0**(-4)*np.array([6,10,40,60,100,400])
# rfr_tune_mss_res5 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'min_samples_split', vals_mss, dpar)
# print([rfr_tune_mss_res0, rfr_tune_mss_res1, rfr_tune_mss_res2, rfr_tune_mss_res3, rfr_tune_mss_res4, rfr_tune_mss_res5])
# vals_mss = 10.0**(-6)*np.array([6,10,30,40,60,70,100,400])
# rfr_tune_mss_res6 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'min_samples_split', vals_mss, dpar)

#tuning max_features; 0.8 seems good
# dpar = {'n_estimators' : 130, 'max_depth' : 16, 'min_samples_split' : 10.0**(-4), 'max_leaf_nodes' : None, 'random_state' : 2,
#         'n_jobs' : 5, 'min_samples_leaf' : 1}
# vals_mf = [0.4, 0.6, 0.8, 'auto']
# rfr_tune_mf_res0 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'max_features', vals_mf, dpar)
# print(rfr_tune_mf_res0)
# vals_mf = [0.65, 0.7, 0.75, 0.85, 0.9, 0.95]
# rfr_tune_mf_res1 = tune_svsp(RandomForestRegressor, xtr0, ytr, 360, 'max_features', vals_mf, dpar)


endtime = how_long(tstart)


#Section B Part 2 version 2

#trying RandomizedSearchCV for adjusting parameters
rfr = RandomForestRegressor(n_estimators=20, max_depth=25, min_samples_split=0.4, max_leaf_nodes=50,
                            max_features=0.3, random_state=2, n_jobs=5, min_samples_leaf=0.15)
sgdr = linear_model.SGDRegressor(max_iter=7, random_state=2, alpha=1e-05)
gbr = GradientBoostingRegressor(learning_rate=0.2, min_samples_leaf=50, max_features='sqrt',
                                subsample=0.8, random_state=2, n_estimators=40, min_samples_split=0.6, max_depth=2)
xgbr = xgb.XGBRegressor(random_state=2, subsample=0.7, max_depth=7)

models = [[rfr, sgdr, gbr, xgbr]]
models.append([copy.copy(m) for m in models[0]])

rfr_param_grid = {'min_samples_leaf' : np.arange(0.05, 0.36, 0.05)}
sgdr_param_grid = {'alpha': 10.0**-np.arange(1,7)}
gbr_param_grid = {'n_estimators' : range(10, 61, 10),
                  'max_depth' : range(2, 8),
                 'min_samples_split' : np.arange(0.5, 0.95, 0.05)}
xgbr_param_grid = {'max_depth' : [3, 5, 6, 7],
                  'subsample' : np.arange(0.4, 1, 0.1)}

par_grids = [rfr_param_grid, sgdr_param_grid, gbr_param_grid, xgbr_param_grid]
best_models = pd.DataFrame(index=range(4), columns=range(2))
predict_tr = best_models.copy()
predict_test = best_models.copy()
cvresults_randsearch = best_models.copy()

#making RandomizedSearhCV
def makegs(model_mgs, pg_mgs):
    return RandomizedSearchCV(model_mgs, param_distributions = pg_mgs, n_iter=9, cv=3, n_jobs=5, random_state=2)

#fitting Randomized Search
def fitgs(gsfgs, xfgs, yfgs):
    gsfgs.fit(xfgs, yfgs)
    return gsfgs

#making predictions
def predbm(bmpbm, xtrpbm, xtestpbm):
    ptr = bmpbm.predict(xtrpbm)
    ptest = bmpbm.predict(xtestpbm)
    return ptr, ptest

tstart = how_long(tstart)

#i=1 is for fewer features set, not i=0
# for j in range(1,4):
#     i=0
#     gsearch = makegs(models[j], par_grids[j])
#     gsearch = fitgs(gsearch, lxtr[i], ytr)
#     tstart = how_long(tstart)
#     print(gsearch.best_params_)
#     for i in range(2):
#         best_models.iloc[j, i] = gsearch.best_estimator_

#without Randomized Search
# for j in range(4):
#     for i in range(2):
#         best_models.iloc[j, i] = models[i][j]
#         best_models.iloc[j, i].fit(lxtr[i], ytr)
#         tstart = how_long(tstart)

#with Randomized Search
# print('Randomized Search CV')
# for i in range(2):
#     print('Data set {}'.format(i))
#     for j in range(4):
#         print('Model {}'.format(j))
#         gsearch = makegs(models[i][j], par_grids[j])
#         gsearch = fitgs(gsearch, lxtr[i], ytr)
#         tstart = how_long(tstart)
#         best_models.iloc[j, i] = gsearch.best_estimator_
#         print(gsearch.best_params_)
#         cvres = pd.DataFrame(gsearch.cv_results_)
#         cvresults_randsearch.iloc[j, i] = cvres.to_string()
#         print(cvres)

# for i in range(2):
#     for j in range(4):
#         prtr, prtest = predbm(best_models.iloc[j, i], lxtr[i], lxtest[i])
#         print(prtr)
#         print(prtest)
#         predict_tr.iloc[j, i] = prtr
#         predict_test.iloc[j, i] = prtest
#         tstart = how_long(tstart)


#Section C: RMSE CV scores

tstart = time.time()

#root mean squared error cross-validation scores
def rmse_cv(model, x_rmsecv, y_rmsecv):
    rmse = np.sqrt(-cross_val_score(model, x_rmsecv, y_rmsecv, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

scores_rmsecv = pd.DataFrame(index=range(4), columns=range(2))

print('RMSE CV scores, fewer features models only')
for j in range(4):
    scores_rmsecv[1][j] = rmse_cv(best_models[1][j], xtr1, ytr)
    print(scores_rmsecv[1][j])

endtime = how_long(tstart)


#Section D: model scores and RMSLE scores

tstart = time.time()

#making some empty dataframes
tr_scores_m = pd.DataFrame(index=range(4), columns=range(2))
tr_scores_rmsle = tr_scores_m.copy()
test_scores_m = tr_scores_m.copy()
test_scores_rmsle = tr_scores_m.copy()

#function for putting scores into dataframes
def scores(model_sc, x_sc, y_sc, predict_sc):
    score_m = model_sc.score(x_sc, y_sc)
    score_rmsle = rmsle(y_sc, predict_sc)
    return score_m, score_rmsle

#going through models and printing scores
for i in range(2):
    print('Set {}'.format(i))
    for j in range(4):
        tr_scores_m[i][j], tr_scores_rmsle[i][j] = scores(best_models[i][j], lxtr[i], ytr, predict_tr[i][j])
        test_scores_m[i][j], test_scores_rmsle[i][j] = scores(best_models[i][j], lxtest[i], ytest, predict_test[i][j])
        print('Model {}'.format(j))
        print('Training {}, {}'.format(tr_scores_m[i][j], tr_scores_rmsle[i][j]))
        print('Test {}, {}'.format(test_scores_m[i][j], test_scores_rmsle[i][j]))

endtime = how_long(tstart)


#Section E: feature importances

tstart = time.time()

#looking at feature importances
def avpr_featimp(smodels, colnames):
    df_f_imp = pd.DataFrame(index=colnames, columns=[0, 2, 3])
    feat_imp = [m.feature_importances_ for m in smodels]
    
    df_f_imp.iloc[:] = np.transpose(feat_imp)
    df_f_imp['average'] = df_f_imp.mean(axis=1)
    print(df_f_imp)
    print(df_f_imp['average'].loc[df_f_imp['average']>0.01])
    return df_f_imp

print('Feature importances')
cm0 = best_models[0].iloc[[0, 2, 3]]
f_imp0 = avpr_featimp(cm0, x0.columns)
cm1 = best_models[1].iloc[[0, 2, 3]]
f_imp1 = avpr_featimp(cm1, x1.columns)

endtime = how_long(tstart)


#Section F: stacking averaged models

#copied from Kaggle kernel "Stacked Regressions to predict House Prices", by Serigne

# class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
#     def __init__(self, models):
#         self.models = models
        
#     # we define clones of the original models to fit the data in
#     def fit(self, X, y):
#         self.models_ = [clone(x) for x in self.models]
        
#         # Train cloned base models
#         for model in self.models_:
#             model.fit(X, y)

#         return self
    
#     #Now we do the predictions for cloned models and average them
#     def predict(self, X):
#         predictions = np.column_stack([
#             model.predict(X) for model in self.models_
#         ])
#         return np.mean(predictions, axis=1)

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X.iloc[train_index], y.iloc[train_index])
                y_pred = instance.predict(X.iloc[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

#end of copied code

# averaged_models = AveragingModels(models = (bests[0], bests[2], bests[3]))
# averaged_models.fit(X_train, y_train)
# pred_av = averaged_models.predict(X_train)
# score_average = rmsle(y_train, pred_av)
# print('Average:')
# print(score_average)

# test_pr_av = averaged_models.predict(X_test)
# score_test_av = rmsle(y_test, test_pr_av)
# print(score_test_av)

# stacked_averaged_models = StackingAveragedModels(base_models = (best_models[0], best_estimators[2]),
#                                                  meta_model = best_estimators[3])
# stacked_averaged_models.fit(X_train, y_train)
# pred_st_av = stacked_averaged_models.predict(X_train)
# score_st_av = rmsle(y_train, pred_st_av)
# print('Stacked average:')
# print(score_st_av)

# test_pr_st_av = stacked_averaged_models.predict(X_test)
# score_test_st_av = rmsle(y_test, test_pr_st_av)
# print(score_test_st_av)

# stacked_averaged_models.fit(dfx, dfy)

# pred_st_av = stacked_averaged_models.predict(dfx)

# score_st_av = rmsle(dfy, pred_st_av)
# print('Stacked average:')
# print(score_st_av)

# predictions_f = stacked_averaged_models.predict(test)
# print(predictions_f)

# df_s = pd.DataFrame(data=testRaw[['User_ID', 'Product_ID']], index=test.index)
# df_s['Purchase'] = predictions_f
# print(df_s)
# df_s.to_csv('bfsubmission1.csv', index=False)